In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from sklearn.model_selection import train_test_split
import tqdm as notebook_tqdm


In [3]:
import pandas as pd
# Load your custom CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\PAVANI\OneDrive\Desktop\IBM Project\dataset\health prescription data.csv")
# Check the dataset to see if there are any issues
print(df.head())


   SUBJECT_ID  ROW_ID  HADM_ID           CATEGORY ADMISSION_TYPE  \
0       26880     178   135453  Discharge summary      EMERGENCY   
1       42130     181   114236  Discharge summary       ELECTIVE   
2       76874     212   113329  Discharge summary      EMERGENCY   
3       66479     220   134640  Discharge summary      EMERGENCY   
4       31502       8   125483  Discharge summary      EMERGENCY   

                                       DIAGNOSIS  \
0                             S/P FALL;TELEMETRY   
1                   LEFT SPHENOID MENENGIOMA/SDA   
2                         TYLENOL BENZO OVERDOSE   
3                              PEDISTRIAN STRUCK   
4   RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE   

                                                TEXT  
0  Admission Date:  [**2162-3-3**]              D...  
1  Admission Date:  [**2150-2-25**]              ...  
2  Admission Date:  [**2101-10-25**]             ...  
3  Admission Date:  [**2148-2-3**]              D...  
4  

In [5]:
train_df, val_df = train_test_split(df, test_size=0.2)


In [15]:
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

# Label Encoding
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['CATEGORY'].tolist())  # Update 'CATEGORY' as needed
val_labels = label_encoder.transform(val_df['CATEGORY'].tolist())  # Update 'CATEGORY' as needed

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Change dtype here to long
        return item

    def __len__(self):
        return len(self.labels)


# Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_texts = train_df['TEXT'].tolist()  # Assuming 'TEXT' column contains the text
val_texts = val_df['TEXT'].tolist()  # Assuming 'TEXT' column contains the text

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Define the datasets with label encoding
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the trainer
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

# Start training
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Found dtype Long but expected Float

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Adjust num_labels based on your dataset


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
    eval_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model every epoch
    load_best_model_at_end=True,     # Load best model when finished training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [12]:
trainer.train()


TypeError: new(): invalid data type 'str'